In [1]:
pip install timm

     |████████████████████████████████| 431 kB 927 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
import gc
import os
import sys
import urllib.request
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

model_type = "DPT_Large"
depth_estimator = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
depth_estimator.to(device)
depth_estimator.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

Downloading: "https://github.com/intel-isl/MiDaS/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt" to /root/.cache/torch/hub/checkpoints/dpt_large-midas-2f21e586.pt


  0%|          | 0.00/1.28G [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [3]:
path = "../input/image-matching-challenge-2022/train/"
lst = os.listdir(path)
if not os.path.exists("depth_maps"):
    os.mkdir("depth_maps")
for _folder in lst:
    if os.path.isdir(path + _folder):
        if not os.path.exists("depth_maps/" + _folder):
            os.mkdir("depth_maps/" + _folder)
        lst2 = os.listdir(path + _folder + "/images/")
        for fname in lst2:
            img = cv2.imread(path + _folder + "/images/" + fname)
            scale = 640 / max(img.shape[0], img.shape[1])
            w = int(img.shape[1] * scale)
            h = int(img.shape[0] * scale)
            img = cv2.resize(img, (w, h))
            input_batch = transform(img).to(device)
            with torch.no_grad():
                mask = depth_estimator(input_batch)

                mask = torch.nn.functional.interpolate(
                    mask.unsqueeze(1),
                    size=img.shape[:2],
                    mode="bicubic",
                    align_corners=False,
                ).squeeze()

            mask = mask.cpu().numpy()
            cv2.imwrite("depth_maps/" + _folder + "/" + fname, mask)
            break

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
